In [1]:
import logging
import math
import os

logging.basicConfig(level=logging.INFO)
logging.info("library loading")
logging.info("DEBUG")

import torch
import librosa

torch.set_grad_enabled(False)

import cached_conv as cc
import gin
import nn_tilde
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from absl import flags, app

import sys, os
import rave
import rave.model
import rave.blocks
import rave.core
import rave.resampler
import IPython.display as ipd

INFO:root:library loading
INFO:root:DEBUG


In [2]:
run = "runs/test_framesynth_c0ad0c727d"

ema_weights = False

cc.use_cached_conv(True)

gin.parse_config_file(os.path.join(run, "config.gin"))
checkpoint = rave.core.search_for_run(run)
print("loading checkpoint:", checkpoint)

pretrained = rave.RAVE()

with open("default.txt", "w") as file:
    for param, val in pretrained.speaker_encoder.state_dict().items():
        file.write(param + "\n")
        file.write(str(val) + "\n")
    
if checkpoint is not None:
    checkpoint = torch.load(checkpoint, map_location='cpu')
    if ema_weights is True and "EMA" in checkpoint["callbacks"]:
        pretrained.load_state_dict(
            checkpoint["callbacks"]["EMA"],
            strict=False,
        )
    else:
        pretrained.load_state_dict(
            checkpoint["state_dict"],
            strict=False,
        )
else:
    print("No checkpoint found, RAVE will remain randomly initialized")

pretrained.eval()

with open("pretrained.txt", "w") as file:
    for param, val in pretrained.speaker_encoder.state_dict().items():
        file.write(param + "\n")
        file.write(str(val) + "\n")

logging.info("warmup pass")
test, _ = librosa.load("libri.wav", sr=44100, mono=True)
test = torch.tensor(test[:2**14]).unsqueeze(0).unsqueeze(0)
#x = torch.rand(1, 1, 2**14)
pretrained(test)

logging.info("optimize model")

for m in pretrained.modules():
    if hasattr(m, "weight_g"):
        nn.utils.remove_weight_norm(m)

logging.info("script model")

loading checkpoint: runs/test_framesynth_c0ad0c727d/version_4/checkpoints/last-v1.ckpt


/opt/tljh/user/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded pretrained speaker encoder


Using cache found in /home/jupyter-arbu/.cache/torch/hub/bshall_hubert_main
INFO:root:warmup pass
INFO:root:optimize model
INFO:root:script model


In [8]:
import matplotlib.pyplot as plt
import scipy.io.wavfile
from  scipy.io import wavfile 

my_pqmf = pretrained.pqmf
my_encoder = pretrained.encoder
my_frame_synth = pretrained.frame_synth
my_speaker_encoder = pretrained.speaker_encoder
my_excitation = pretrained.excitation_module
wavenet = pretrained.wavenet

emb_audio, _ = librosa.load("../vctk-small/p228/p228_005_mic1.flac", sr=44100, mono=True)
emb_audio = torch.tensor(emb_audio[:131072]).unsqueeze(0)
emb_audio_pqmf = my_pqmf(torch.tensor(emb_audio.unsqueeze(1)))
target_emb = my_speaker_encoder(emb_audio_pqmf).unsqueeze(2)

x, _ = librosa.load("libri.wav", sr=44100, mono=True)
x = torch.tensor(x[:2*131072]).unsqueeze(0)

x_multi = my_pqmf(x.unsqueeze(1))

z = my_encoder(x_multi[:, :6, :])
emb = target_emb.repeat(z.shape[0], 1, z.shape[-1])

print(z.shape, emb.shape)
        
#z = torch.cat((z, emb), dim=1)
frames = my_frame_synth(z, emb)

#ex = my_excitation(x, 1.0)
#ipd.display(ipd.Audio(ex[0].numpy(), rate=44100))
ex = my_excitation(x, 2.0)
ipd.display(ipd.Audio(ex[0].numpy(), rate=44100))
#y_multiband = my_decoder(z)

interp = torch.nn.functional.interpolate(
            frames, size=ex.shape[-1], mode="linear"
        )

y = wavenet(ex, interp)

ipd.display(ipd.Audio(y[0].numpy(), rate=44100))

/tmp/ipykernel_1183477/1369637926.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb_audio_pqmf = my_pqmf(torch.tensor(emb_audio.unsqueeze(1)))


torch.Size([1, 64, 256]) torch.Size([1, 256, 256])
